## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

## Download

Retrieve the page

In [4]:
url = "https://itwebservices.placer.ca.gov/coviddashboard/#cases-by-zip-coderegion"

In [5]:
page = requests.get(url, verify=False)

ConnectionError: HTTPSConnectionPool(host='itwebservices.placer.ca.gov', port=443): Max retries exceeded with url: /coviddashboard/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f5f304b8be0>: Failed to establish a new connection: [Errno 110] Connection timed out'))

## Parse

In [13]:
soup = BeautifulSoup(page.content, "html.parser")

Find script tag with the Infogram data

In [14]:
for elem in soup(text=re.compile(r"Foresthill")):
    script_content = elem

In [15]:
data = json.loads(script_content)

In [16]:
zip_cities_list = data["x"]["calls"][1]["args"][-3]

In [17]:
dict_list = []

In [18]:
TAG_RE = re.compile(r"<[^>]+>")

In [19]:
for z in zip_cities_list:
    split_zips = z.split("<br/>")
    area = TAG_RE.sub("", str(split_zips[0]))
    zip_code, city = area.split(" - ")
    clean_cases = re.sub("Number of cases: ".lower(), "", str(split_zips[1]).lower())
    d = dict(area=area, city=city, zip_code=zip_code, confirmed_cases=clean_cases)
    dict_list.append(d)

In [20]:
df = pd.DataFrame(dict_list)

Get timestamp

In [21]:
time_div = soup.find("div", id="dashboard-data-last-updated")

In [22]:
timestamp = time_div.find("span", class_="value-output").get_text()

In [23]:
latest_date = pd.to_datetime(timestamp).date()

In [24]:
df["county_date"] = latest_date

In [25]:
df.insert(0, "county", "Placer")

Clean up

In [26]:
df["area"] = df["area"].str.replace(" - ", ": ")

In [27]:
df = df[["county", "area", "confirmed_cases", "county_date", "zip_code"]].rename(
    columns={"zip_code": "zip"}
)

## Vet

In [29]:
try:
    assert not len(df) < 32
except AssertionError:
    raise AssertionError("Placer County's scraper is missing rows")

In [30]:
try:
    assert not len(df) > 32
except AssertionError:
    raise AssertionError("Placer County's scraper has more rows than before")

## Export

Set date

In [31]:
tz = pytz.timezone("America/Los_Angeles")

In [32]:
today = datetime.now(tz).date()

In [33]:
slug = "placer"

In [34]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [35]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [36]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [37]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [38]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)